In [1]:
%load_ext autoreload
%autoreload 2

# Aggregate pairs and compute counts, probabilities, and similarities
I don't think the probabilities calculated here are actually used because we re-calculate them after filtering.

In [2]:
from os.path import join

from mpire import WorkerPool
import pandas as pd
from pandarallel import pandarallel

from nama.data.filesystem import glob, download_file_from_s3, save_file
from nama.data.match import levenshtein_similarity

In [3]:
# DO FOR BOTH GIVEN AND SURNAME
given_surname = "given"
# given_surname = "surname"

in_path = f"s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-{given_surname}-pairs/"
out_path = f"s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-{given_surname}-aggr-v2.parquet"

In [4]:
# read input files into dataframe array
def read_file(filename):
    filename = download_file_from_s3(filename) if filename.startswith("s3://") else filename
    return pd.read_parquet(filename)

with WorkerPool() as pool:
    dfs = pool.map(read_file, glob(join(in_path, "part-*")), progress_bar=True)
print(len(dfs))

Exception occurred, terminating ... :  23%|██▎       | 2704/12000 [02:47<08:21, 18.52it/s]

Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02781.parquet from S3: Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02701.parquet from S3: 

Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02645.parquet from S3: 
Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02801.parquet from S3: 


Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02722.parquet from S3: 
Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02734.parquet from S3: 
Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02747.parquet from S3: 
Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02691.parquet from S3: 
Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02680.parquet from S3: 
Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02669.parquet from S3: 
Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02791.parquet from S3: Error downloading file s3://fs-nama-data/2024/familysearch-names/interim/tree-hr-given-pairs/part-02635.parquet from S3: 

Error downloading file 

Exception in thread Thread-27 (_terminate_worker):
Traceback (most recent call last):
  File "/home/dallan/.local/share/mise/installs/python/3.12.7/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-28 (_terminate_worker):
Traceback (most recent call last):
  File "/home/dallan/.local/share/mise/installs/python/3.12.7/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-29 (_terminate_worker):
Traceback (most recent call last):
  File "/home/dallan/.local/share/mise/installs/python/3.12.7/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/home/dallan/rootsdev/nama/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    self.run()
  File "/home/dallan/rootsdev/nama/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/home/dallan/.local/share/mise/installs/python/3.12.7/lib/python3

KeyboardInterrupt: 

In [ ]:
df = pd.concat(dfs)
del dfs
print(df.shape)
print(df)

In [ ]:
# group by name and alt name and calculate frequency
df["frequency"] = 1
grouped = df.groupby(["name", "alt_name"]).sum().reset_index()
del df
print(grouped.shape)

## Calculate ordered and unordered probabilities

In [ ]:
# sum frequency by name
sum_name_freq = (
    grouped.groupby('name')['frequency']
    .sum()
    .reset_index()
    .rename(columns={'frequency': 'sum_frequency'})
)

In [ ]:
# sum frequency by alt_name
sum_alt_name_freq = (
    grouped.groupby('alt_name')['frequency']
    .sum()
    .reset_index()
    .rename(columns={'alt_name': 'name', 'frequency': 'sum_frequency'})
)

In [ ]:
# total frequency for name or alt_name
total_name_freq = (
    pd.concat([sum_name_freq, sum_alt_name_freq]).groupby("name").sum().reset_index()
)

In [ ]:
# include reverse name-alt_name frequency
grouped = (
    pd.merge(
        grouped,
        grouped,
        how="left",
        left_on=["name", "alt_name"],
        right_on=["alt_name", "name"],
        suffixes=("", "_ignore"),
    )
    .drop(columns=["name_ignore", "alt_name_ignore"])
    .rename(columns={"frequency_ignore": "reverse_frequency"})
    .reset_index(drop=True)
    .fillna(0)
)

In [ ]:
# include sum frequency for name
grouped = (
    pd.merge(grouped, sum_name_freq, how="inner", on="name")
    .rename(columns={"sum_frequency": "sum_name_frequency"})
    .reset_index(drop=True)
)

In [ ]:
# include total name frequency
grouped = (
    pd.merge(grouped, total_name_freq, how="inner", on="name")
    .rename(columns={"sum_frequency": "total_name_frequency"})
    .reset_index(drop=True)
)

In [ ]:
# include total alt name frequency
grouped = (
    pd.merge(
        grouped,
        total_name_freq.rename(columns={"name": "alt_name"}),
        how="inner",
        on="alt_name",
    )
    .rename(columns={"sum_frequency": "total_alt_name_frequency"})
    .reset_index(drop=True)
)

In [ ]:
# ordered probability = frequency / sum frequency for name
grouped["ordered_prob"] = grouped["frequency"] / grouped["sum_name_frequency"]

In [ ]:
# unordered probability = frequency + reverse frequency / total number of times name and alt_name appear anywhere
#   which is total name frequency + total alt name frequency, but then we've double-counted the times they appear together,
#   so we need to subtract frequency + reverse frequency from the denominator
grouped["unordered_prob"] = (grouped["frequency"] + grouped["reverse_frequency"]) / (
    grouped["total_name_frequency"]
    + grouped["total_alt_name_frequency"]
    - (grouped["frequency"] + grouped["reverse_frequency"])
)

In [ ]:
# calculate levenshtein similarity
pandarallel.initialize(progress_bar=True)
grouped["similarity"] = grouped.parallel_apply(lambda row: levenshtein_similarity(row["name"], row["alt_name"]), axis=1)

In [ ]:
# downgrade types to save space
grouped = grouped.astype(
    {
        "frequency": "int32",
        "reverse_frequency": "int32",
        "sum_name_frequency": "int32",
        "total_name_frequency": "int32",
        "total_alt_name_frequency": "int32",
        "ordered_prob": "float32",
        "unordered_prob": "float32",
        "similarity": "float32",
    }
)

In [ ]:
# write to parquet
save_file(out_path,
          lambda local_out_path : grouped.to_parquet(local_out_path, index=False))

In [ ]:
print(grouped.shape)

In [ ]:
grouped.memory_usage(deep=True)